In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-08-24"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
102,2024-08-24,Filipinas Copa Dos Governadores,07:00,Rain Or Shine Paint.,Ginebra Kings,2.32,1.51,209.5,1.83,1.83,2.5,2.04,1.38,hnPi0cnn,0.431034,0.662252,0.546448,0.546448,0.093286,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.299090,0.000000,0.272918,0.0,0.0,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
103,2024-08-24,Mundo Copa Do Mundo Feminina,20:00,México F,Republica Checa F,8.20,1.07,135.5,1.87,1.91,12.5,1.99,1.01,SlTB5oet,0.121951,0.934579,0.534759,0.523560,0.056531,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,157.62,0.00,0.0,0.0,0.0,0.0,1.087739,0.014965,0.461976,0.0,0.0,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
104,2024-08-24,Tailândia Tbl,04:00,Chaophraya Thunder,Bangkok Tiger Thunder,2.55,1.49,166.5,1.83,1.83,-3.5,1.83,1.71,EsL4LQX7,0.392157,0.671141,0.546448,0.546448,0.063298,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.371056,0.000000,0.047939,0.0,0.0,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
105,2024-08-24,México Lnbp,22:00,Correcaminos,Abejas,1.92,1.80,173.5,1.85,1.85,-1.5,2.05,1.92,0jNEhXg3,0.520833,0.555556,0.540541,0.540541,0.076389,0.0,0.6,NaN,NaN,NaN,NaN,NaN,163.968,32.610258,0.198882,0.0,164.01,209.04,209.44,0.0,0.0,0.0,0.0,0.045620,0.000000,0.046309,0.0,0.0,inf,0.0,0.0,0.0,-3.27,-0.654,-1.223242,0.0,0.0,0.0
106,2024-08-24,México Lnbp,23:00,Freseros,Soles,2.10,1.68,168.5,1.81,1.89,-3.5,1.95,2.00,hbuMjBOF,0.476190,0.595238,0.552486,0.529101,0.071429,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.00,226.00,0.0,0.0,0.0,0.0,0.157135,0.030578,0.017901,0.0,0.0,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,22:00,México Lnbp,Correcaminos,Abejas,173.5,1.85,0.9997,Over
1,23:00,México Lnbp,Freseros,Soles,168.5,1.81,0.9531,Over
